## 심장마비 확률이 높은사람?
- 성별, 나이, 혈압, 콜레스테롤, 공복혈당, 최대 심박수 등의 컬럼이 있음
- 평가: ROC-AUC, 정확도(Accuracy), F1 을 구하시오
- target : output (1:심장마비 확률 높음, 0:심장마비 확률 낮음)
- csv파일 생성 : 수험번호.csv (예시 아래 참조)
~~~
id,output
41,0.633
28,0.123
222,0.355
~~~



## 데이터 불러오기

In [1]:
import pandas as pd
train = pd.read_csv('heart/2files/train.csv')
test = pd.read_csv('heart/2files/test.csv')

##EDA

In [2]:
print(train.shape, test.shape)

(242, 15) (61, 14)


In [3]:
display(train.head(2)) # 타깃: output, 분류
test.head(2)

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,48,53,0,2,128,216,0,0,115,0,0.0,2,0,0,1
1,124,39,0,2,94,199,0,1,179,0,0.0,2,0,2,1


,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,41,48,1,1,130,245,0,0,180,0,0.2,1,0,2
1,28,65,0,2,140,417,1,0,157,0,0.8,2,1,2


In [6]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        242 non-null    int64  
 1   age       242 non-null    int64  
 2   sex       242 non-null    int64  
 3   cp        242 non-null    int64  
 4   trtbps    242 non-null    int64  
 5   chol      242 non-null    int64  
 6   fbs       242 non-null    int64  
 7   restecg   242 non-null    int64  
 8   thalachh  242 non-null    int64  
 9   exng      242 non-null    int64  
 10  oldpeak   242 non-null    float64
 11  slp       242 non-null    int64  
 12  caa       242 non-null    int64  
 13  thall     242 non-null    int64  
 14  output    242 non-null    int64  
dtypes: float64(1), int64(14)
memory usage: 28.5 KB
None


In [7]:
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        61 non-null     int64  
 1   age       61 non-null     int64  
 2   sex       61 non-null     int64  
 3   cp        61 non-null     int64  
 4   trtbps    61 non-null     int64  
 5   chol      61 non-null     int64  
 6   fbs       61 non-null     int64  
 7   restecg   61 non-null     int64  
 8   thalachh  61 non-null     int64  
 9   exng      61 non-null     int64  
 10  oldpeak   61 non-null     float64
 11  slp       61 non-null     int64  
 12  caa       61 non-null     int64  
 13  thall     61 non-null     int64  
dtypes: float64(1), int64(13)
memory usage: 6.8 KB
None


In [8]:
train.describe()

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.00000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000
mean,154.706612,54.516529,0.673554,0.913223,131.070248,246.933884,0.119835,0.53719,149.590909,0.347107,1.019835,1.380165,0.785124,2.347107,0.520661
std,86.328576,9.021513,0.469885,1.020897,17.716978,52.236414,0.325441,0.53183,21.763201,0.477037,1.182137,0.621285,1.052218,0.607160,0.500608
min,0.000000,34.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.00000,88.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,81.250000,47.250000,0.000000,0.000000,120.000000,212.000000,0.000000,0.00000,133.250000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,158.500000,55.000000,1.000000,1.000000,130.000000,243.000000,0.000000,1.00000,152.000000,0.000000,0.600000,1.000000,0.000000,2.000000,1.000000
75%,228.750000,61.000000,1.000000,2.000000,140.000000,274.000000,0.000000,1.00000,165.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,302.000000,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.00000,195.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [9]:
test.describe()

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
count,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000,61.000000
mean,136.295082,53.770492,0.721311,1.180328,133.819672,243.606557,0.262295,0.491803,149.868852,0.245902,1.118033,1.475410,0.508197,2.180328
std,91.792764,9.370865,0.452075,1.056854,16.770518,50.523684,0.443533,0.504082,27.170128,0.434194,1.079121,0.594602,0.868388,0.619360
min,1.000000,29.000000,0.000000,0.000000,105.000000,168.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,64.000000,48.000000,0.000000,0.000000,120.000000,204.000000,0.000000,0.000000,138.000000,0.000000,0.000000,1.000000,0.000000,2.000000
50%,133.000000,56.000000,1.000000,1.000000,132.000000,235.000000,0.000000,0.000000,157.000000,0.000000,1.000000,2.000000,0.000000,2.000000
75%,206.000000,59.000000,1.000000,2.000000,144.000000,277.000000,1.000000,1.000000,170.000000,0.000000,1.600000,2.000000,1.000000,3.000000
max,300.000000,69.000000,1.000000,3.000000,180.000000,417.000000,1.000000,1.000000,202.000000,1.000000,4.000000,2.000000,4.000000,3.000000


In [13]:
print(train.isnull().sum())

id          0
age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
dtype: int64


In [12]:
print(test.isnull().sum())

id          0
age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
dtype: int64


## 데이터 전처리 & 피처엔지니어링

In [10]:
target = train.pop('output')

## 검증 데이터 분리

In [14]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train, target, test_size = 0.15, random_state=42)

In [15]:
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

(205, 14) (37, 14) (205,) (37,)


## 모델 & 평가

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
rf = RandomForestClassifier(random_state=42)
rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
pred_proba = rf.predict_proba(X_val)
print("정확도:", accuracy_score(y_val, pred))
print("F1:", f1_score(y_val, pred))
print("Roc:", roc_auc_score(y_val, pred_proba[:, 1]))

정확도: 1.0
F1: 1.0
Roc: 1.0


In [18]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
xgb = XGBClassifier(random_state=42)
xgb.fit(X_tr, y_tr)
pred = xgb.predict(X_val)
pred_proba = xgb.predict_proba(X_val)
print("정확도:", accuracy_score(y_val, pred))
print("F1:", f1_score(y_val, pred))
print("Roc:", roc_auc_score(y_val, pred_proba[:, 1]))

정확도: 0.972972972972973
F1: 0.9696969696969697
Roc: 0.9761904761904762


In [19]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
xgb = LGBMClassifier(random_state=42, verbose= -1)
xgb.fit(X_tr, y_tr)
pred = xgb.predict(X_val)
pred_proba = xgb.predict_proba(X_val)
print("정확도:", accuracy_score(y_val, pred))
print("F1:", f1_score(y_val, pred))
print("Roc:", roc_auc_score(y_val, pred_proba[:, 1]))

정확도: 1.0
F1: 1.0
Roc: 1.0


## 예측 및 csv 제출

In [26]:
pred_proba = xgb.predict_proba(test)
submit = pd.DataFrame(
    {
        'pred': pred_proba[:, 1]
    }
)
submit.to_csv('연습문제3번제출.csv', index=False)

##정리

In [27]:
y_test = pd.read_csv("heart/2files/y_test.csv")
pred = pd.read_csv("연습문제3번제출.csv")
print(roc_auc_score(y_test, pred))

1.0
